In [12]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
import datetime
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [13]:
data_orders = pd.read_excel(io='data/cafecultura_pos.xlsx', sheetname='pos_orders')

In [14]:
data_orders = data_orders.drop(['DISCOUNT_PROC', 'DISCOUNT_TOTAL', 'CUSTOMER_ID', 'CASH_RECEIVED', 'CHANGE_BACK','COMMENT',   \
                               'TAGS', 'INOUT', 'DRAFT',  'SERVICE_TIME'], axis=1)

In [15]:
data_cart = pd.read_excel(io='data/cafecultura_pos.xlsx', sheetname='pos_cart')

In [16]:
data_cart = data_cart.drop(['ID', 'CUSTOMER_ID', 'SESSION_ID'], axis=1)

In [17]:
items_str = []
for i in range(1, 34):
    items_str.append('item_' + str(i))

In [18]:
df_cols = ['ID','Seller', 'Total_sum', 'Entry_time', 'Num_guests', 'Num_items', 'Duration']
df_cols.extend(items_str)

In [19]:
final_df  = pd.DataFrame(columns=df_cols)

In [20]:
def row_creation(order, cart, ID, frame):
    ID = ID*1.0
    if (ID not in frame.ID.tolist()):
        temp_series = pd.Series(index = df_cols)
        temp_series['ID'] = ID
        temp_series['Seller'] = order['SELLER_ID']
        temp_series['Total_sum'] = order['AMOUNT_TOTAL']
        temp_series['Entry_time'] = cart['ADDED']
        temp_series['Num_guests'] = order['GUESTS']
        temp_series['Num_items'] = cart['ITEM_QTY']
        temp_series['Duration'] = np.round((parse(order['ADDED']) - parse(cart['ADDED'])).seconds/60, decimals=1)
        
        
        for i in range(1, 34):
            temp_series['item_'+str(i)] = 0
            
        n_item = cart['ITEM_ID']
        temp_series['item_' + str(n_item)] = 1
        
        
        frame = frame.append(temp_series, ignore_index=True)
    else:
        frame.loc[frame[frame.ID==ID].index, 'Num_items'] = frame.loc[frame[frame.ID==ID].index,\
                                                                                  'Num_items'] + cart['ITEM_QTY']
        
        n_item = cart['ITEM_ID']
        frame.loc[frame[frame.ID==ID].index, 'item_' + str(n_item)] = frame.loc[frame[frame.ID==ID].index,\
                                                                                'item_' + str(n_item)] + 1
        
        frame.loc[frame[frame.ID==ID].index, 'Duration'] =  np.round((parse(order['ADDED']) - \
                                                                     parse(frame.loc[frame[frame.ID==ID].index[0], \
                                                                                     'Entry_time'])).seconds/60, decimals = 1)
                                
    return frame
        
        

In [21]:
for i in range(data_orders.shape[0]):
    temp_id = data_orders.ID[i]
    for j in range(data_cart.shape[0]):
        if (temp_id==data_cart.ORDER_ID[j]):
            final_df =  row_creation(data_orders.loc[i, :], data_cart.loc[j, :], temp_id, final_df)

In [25]:
final_df = final_df.drop(final_df[final_df.Duration>180].index, axis=0)
final_df.Duration[final_df.Duration<2] = 2

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
final_df = final_df.reset_index(drop= True)
for i in range(final_df.shape[0]):
    dt = parse(final_df.loc[i, 'Entry_time'])
    final_df.loc[i, 'Date'] = dt.date()
    final_df.loc[i, 'Time'] = dt.time()

In [26]:
t1 = datetime.time(8,0,0)
t2 = datetime.time(13,0,0)
t3 = datetime.time(17,0,0)
t4 = datetime.time(23,0,0)

In [27]:
for i in range(final_df.shape[0]):
    t = final_df.loc[i, 'Time']
    if ((t > t1) & (t < t2)):
        final_df.loc[i, 'T_period'] = 'morning'
        
    if ((t > t2) & (t < t3)):
        final_df.loc[i, 'T_period'] = 'midday'
        
    if ((t > t3) & (t < t4)):
        final_df.loc[i, 'T_period'] = 'evening'
    

In [28]:
final_df = final_df.drop(['Entry_time', 'Time'], axis=1)

In [29]:
for i in range(final_df.shape[0]):
    dt = final_df.loc[i, 'Date']
    final_df.loc[i, 'Weekday'] = dt.isoweekday()

In [33]:
final_df.head(5)

,ID,Seller,Total_sum,Num_guests,Num_items,Duration,item_1,item_2,item_3,item_4,...,item_27,item_28,item_29,item_30,item_31,item_32,item_33,Date,T_period,Weekday
0,185.0,1.0,4.5,1,2,2.0,0,0,0,0,...,0,0,0,0,0,0,0,2017-04-06,midday,4.0
1,150.0,2.0,11.0,1,4,31.2,0,0,0,0,...,0,0,0,0,0,0,0,2017-01-24,evening,2.0
2,156.0,1.0,2.5,1,1,2.0,0,0,1,0,...,0,0,0,0,0,0,0,2017-04-06,morning,4.0
3,184.0,1.0,4.0,1,1,16.9,0,0,0,0,...,0,0,0,0,0,0,0,2017-04-06,midday,4.0
4,157.0,1.0,2.0,1,1,2.0,0,0,0,1,...,0,0,0,0,0,0,0,2017-04-06,morning,4.0


In [34]:
final_df = final_df.drop([1], axis=0)

In [35]:
final_df.to_csv(path_or_buf='data/first_df.csv', index = False)

In [37]:
final_df.to_excel(excel_writer='data/first_df.xlsx', index = False)